# 다중 벡터저장소 검색기(MultiVectorRetriever)

LangChain에서는 문서를 다양한 상황에서 효율적으로 쿼리할 수 있는 특별한 기능, 바로 `MultiVectorRetriever`를 제공합니다. 이 기능을 사용하면 문서를 여러 벡터로 저장하고 관리할 수 있어, 정보 검색의 정확도와 효율성을 대폭 향상시킬 수 있습니다.

`MultiVectorRetriever`를 활용해 문서당 여러 벡터를 생성하는 몇 가지 방법을 살펴보겠습니다.

**문서당 여러 벡터 생성 방법 소개**

1. **작은 청크 생성**: 문서를 더 작은 단위로 나눈 후, 각 청크에 대해 별도의 임베딩을 생성합니다. 이 방식을 사용하면 문서의 특정 부분에 좀 더 세심한 주의를 기울일 수 있습니다. 이 과정은 `ParentDocumentRetriever`를 통해 구현할 수 있어, 세부 정보에 대한 탐색이 용이해집니다.

2. **요약 임베딩**: 각 문서의 요약을 생성하고, 이 요약으로부터 임베딩을 만듭니다. 이 요약 임베딩은 문서의 핵심 내용을 신속하게 파악하는 데 큰 도움이 됩니다. 문서 전체를 분석하는 대신 핵심적인 요약 부분만을 활용하여 효율성을 극대화할 수 있습니다.

3. **가설 질문 활용**: 각 문서에 대해 적합한 가설 질문을 만들고, 이 질문에 기반한 임베딩을 생성합니다. 특정 주제나 내용에 대해 깊이 있는 탐색을 원할 때 이 방법이 유용합니다. 가설 질문은 문서의 내용을 다양한 관점에서 접근하게 해주며, 더 광범위한 이해를 가능하게 합니다.

4. **수동 추가 방식**: 사용자가 문서 검색 시 고려해야 할 특정 질문이나 쿼리를 직접 추가할 수 있습니다. 이 방법을 통해 사용자는 검색 과정에서 보다 세밀한 제어를 할 수 있으며, 자신의 요구 사항에 맞춘 맞춤형 검색이 가능해집니다.

## 실습에 활용한 문서

소프트웨어정책연구소(SPRi) - 2023년 12월호

- 저자: 유재흥(AI정책연구실 책임연구원), 이지수(AI정책연구실 위촉연구원)
- 링크: https://spri.kr/posts/view/23669
- 파일명: `SPRI_AI_Brief_2023년12월호_F.pdf`

**참고**: 위의 파일은 `data` 폴더 내에 다운로드 받으세요

In [1]:
import os

os.environ["OPENAI_API_KEY"] = ""

os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "10-03"

텍스트 파일에서 데이터를 로드하고, 로드된 문서들을 지정된 크기로 분할하는 전처리 과정을 수행합니다.

분할된 문서들은 추후 벡터화 및 검색 등의 작업에 사용될 수 있습니다.


In [5]:
!pip install -qU langchain_community langchain_openai langchain_chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 64.5 MB/s eta 0:00:00
 

In [9]:
!pip install -qU pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 69.2 MB/s eta 0:00:00


In [10]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("/content/appendix-keywords.txt")
docs = loader.load()

데이터로부터 로드한 원본 도큐먼트는 `docs` 변수에 담았습니다.


In [11]:
print(docs[5].page_content[:500])

판다스 (Pandas)
정의: 판다스는 파이썬 프로그래밍 언어를 위한 데이터 분석 및 조
작 도구를 제공하는 라이브러리입니다. 이는 데이터 분석 작업을
효율적으로 수행할 수 있게 합니다.
예시: 판다스를 사용하여 CSV 파일을 읽고, 데이터를 정제하며,
다양한 분석을 수행할 수 있습니다.
연관키워드: 데이터 분석, 파이썬, 데이터 처리
GPT (Generative Pretrained Transformer)
정의: GPT는 대규모의 데이터셋으로 사전 훈련된 생성적 언어 모델
로, 다양한 텍스트 기반 작업에 활용됩니다. 이는 입력된 텍스트에
기반하여 자연스러운 언어를 생성할 수 있습니다.
예시: 사용자가 제공한 질문에 대해 자세한 답변을 생성하는 챗봇
은 GPT 모델을 사용할 수 있습니다.
연관키워드: 자연어 처리, 텍스트 생성, 딥러닝
InstructGPT
정의: InstructGPT는 사용자의 지시에 따라 특정한 작업을 수행
하기 위해 최적화된 GPT 모델입니다. 이 모델은 보다 정


## Chunk + 원본 문서 검색

대용량 정보를 검색하는 경우, 더 작은 단위로 정보를 임베딩하는 것이 유용할 수 있습니다.

`MultiVectorRetriever` 를 통해 문서를 여러 벡터로 저장하고 관리할 수 있습니다.

`docstore` 에 원본 문서를 저장하고, `vectorstore` 에 임베딩된 문서를 저장합니다.

이로써 문서를 더 작은 단위로 나누어 더 정확한 검색이 가능해집니다. 때에 따라서는 원본 문서의 내용을 조회할 수 있습니다.

In [12]:
# 자식 청크를 인덱싱하는 데 사용할 벡터 저장소
import uuid
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers.multi_vector import MultiVectorRetriever

vectorstore = Chroma(
    collection_name="small_bigger_chunks",
    embedding_function=OpenAIEmbeddings(model="text-embedding-3-small"),
)
# 부모 문서의 저장소 계층
store = InMemoryStore()

id_key = "doc_id"

# 검색기 (시작 시 비어 있음)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

# 문서 ID를 생성합니다.
doc_ids = [str(uuid.uuid4()) for _ in docs]

# 두개의 생성된 id를 확인합니다.
doc_ids

['c7a853ae-5d73-4282-9844-69ec2c9287d2',
 'a9556ca6-261a-4004-b4b3-19fadfa7436e',
 '6d50ebe5-9cbb-4bb0-8efc-aa6b317e60d0',
 '5c808bb6-fb35-4974-96c2-936389f30305',
 '81a15d38-5423-44a5-ad1f-315051a90405',
 '736523c5-246d-44ef-a2d4-10023628eae4',
 '87ad5548-f271-4fcd-9be8-086c42b34cea']

여기서 큰 청크로 분할하기 위한 `parent_text_splitter`

더 작은 청크로 분할하기 위한 `child_text_splitter` 를 정의합니다.


In [13]:
# RecursiveCharacterTextSplitter 객체를 생성합니다.
parent_text_splitter = RecursiveCharacterTextSplitter(chunk_size=600)

# 더 작은 청크를 생성하는 데 사용할 분할기
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

더 큰 Chunk인 Parent 문서를 생성합니다.

In [14]:
parent_docs = []

for i, doc in enumerate(docs):
    # 현재 문서의 ID를 가져옵니다.
    _id = doc_ids[i]
    # 현재 문서를 하위 문서로 분할
    parent_doc = parent_text_splitter.split_documents([doc])

    for _doc in parent_doc:
        # metadata에 문서 ID 를 저장
        _doc.metadata[id_key] = _id
    parent_docs.extend(parent_doc)

`parent_docs` 에 기입된 `doc_id` 를 확인합니다.

In [15]:
# 생성된 Parent 문서의 메타데이터를 확인합니다.
parent_docs[0].metadata

{'producer': '',
 'creator': '',
 'creationdate': '',
 'source': '/content/appendix-keywords.txt',
 'file_path': '/content/appendix-keywords.txt',
 'total_pages': 7,
 'format': 'Tex',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'moddate': '',
 'trapped': '',
 'encryption': '',
 'page': 0,
 'doc_id': 'c7a853ae-5d73-4282-9844-69ec2c9287d2'}

상대적으로 더 작은 Chunk인 Child 문서를 생성합니다.

In [16]:
child_docs = []
for i, doc in enumerate(docs):
    # 현재 문서의 ID를 가져옵니다.
    _id = doc_ids[i]
    # 현재 문서를 하위 문서로 분할
    child_doc = child_text_splitter.split_documents([doc])
    for _doc in child_doc:
        # metadata에 문서 ID 를 저장
        _doc.metadata[id_key] = _id
    child_docs.extend(child_doc)

`child_docs` 에 기입된 `doc_id` 를 확인합니다.

In [17]:
# 생성된 Child 문서의 메타데이터를 확인합니다.
child_docs[0].metadata

{'producer': '',
 'creator': '',
 'creationdate': '',
 'source': '/content/appendix-keywords.txt',
 'file_path': '/content/appendix-keywords.txt',
 'total_pages': 7,
 'format': 'Tex',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'moddate': '',
 'trapped': '',
 'encryption': '',
 'page': 0,
 'doc_id': 'c7a853ae-5d73-4282-9844-69ec2c9287d2'}

각각 분할된 청크의 수를 확인합니다.


In [18]:
print(f"분할된 parent_docs의 개수: {len(parent_docs)}")
print(f"분할된 child_docs의 개수: {len(child_docs)}")

분할된 parent_docs의 개수: 13
분할된 child_docs의 개수: 129


벡터저장소에 새롭게 생성한 작게 쪼개진 하위문서 집합을 추가합니다.

다음으로는 상위 문서는 생성한 UUID 와 맵핑하여 `docstore` 에 추가합니다.

- `mset()` 메서드를 통해 문서 ID와 문서 내용을 key-value 쌍으로 문서 저장소에 저장합니다.

In [19]:
# 벡터 저장소에 parent + child 문서를 추가
retriever.vectorstore.add_documents(parent_docs)
retriever.vectorstore.add_documents(child_docs)

# docstore 에 원본 문서를 저장
retriever.docstore.mset(list(zip(doc_ids, docs)))

유사도 검색을 수행합니다. 가장 유사도가 높은 첫 번째 문서 조각을 출력합니다.

여기서 `retriever.vectorstore.similarity_search` 메서드는 child + parent 문서 chunk 내에서 검색을 수행합니다.

In [20]:
# vectorstore의 유사도 검색을 수행합니다.
relevant_chunks = retriever.vectorstore.similarity_search(
    "삼성전자가 만든 생성형 AI 의 이름은?"
)
print(f"검색된 문서의 개수: {len(relevant_chunks)}")

검색된 문서의 개수: 4


In [21]:
for chunk in relevant_chunks:
    print(chunk.page_content, end="\n\n")
    print(">" * 100, end="\n\n")

FAISS (Facebook AI Similarity Search)
정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리
로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수
있도록 설계되었습니다.
예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾
는 데 FAISS가 사용될 수 있습니다.

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

련성 높은 결과를 생성하도록 설계되었습니다.
예시: 사용자가 "이메일 초안 작성"과 같은 특정 지시를 제공하면,
InstructGPT는 관련 내용을 기반으로 이메일을 작성합니다.
연관키워드: 인공지능, 자연어 이해, 명령 기반 처리
Keyword Search
정의: 키워드 검색은 사용자가 입력한 키워드를 기반으로 정보를

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

연관키워드: 자연어 처리, 딥러닝, 텍스트 생성
FAISS (Facebook AI Similarity Search)
정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리
로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수
있도록 설계되었습니다.
예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모
델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이
쉽게 NLP 작업을 수행할 수 있도록 돕습니다.
예시: HuggingFace의 Transformers 라

이번에는 `retriever.invoke()` 메서드를 사용하여 쿼리를 실행합니다.

`retriever.invoke()` 메서드는 원본 문서의 전체 내용을 검색합니다.

In [22]:
relevant_docs = retriever.invoke("삼성전자가 만든 생성형 AI 의 이름은?")
print(f"검색된 문서의 개수: {len(relevant_docs)}", end="\n\n")
print("=" * 100, end="\n\n")
print(relevant_docs[0].page_content)

검색된 문서의 개수: 3


연관키워드: 자연어 처리, 딥러닝, 텍스트 생성
FAISS (Facebook AI Similarity Search)
정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리
로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수
있도록 설계되었습니다.
예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾
는 데 FAISS가 사용될 수 있습니다.
연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화
Open Source
정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용,
수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신
을 촉진하는 데 중요한 역할을 합니다.
예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.
연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업
Structured Data
정의: 구조화된 데이터는 정해진 형식이나 스키마에 따라 조직된
데이터입니다. 이는 데이터베이스, 스프레드시트 등에서 쉽게 검색
하고 분석할 수 있습니다.
예시: 관계형 데이터베이스에 저장된 고객 정보 테이블은 구조화된
데이터의 예입니다.
연관키워드: 데이터베이스, 데이터 분석, 데이터 모델링
Parser
정의: 파서는 주어진 데이터(문자열, 파일 등)를 분석하여 구조화
된 형태로 변환하는 도구입니다. 이는 프로그래밍 언어의 구문 분
석이나 파일 데이터 처리에 사용됩니다.
예시: HTML 문서를 구문 분석하여 웹 페이지의 DOM 구조를 생성하
는 것은 파싱의 한 예입니다.
연관키워드: 구문 분석, 컴파일러, 데이터 처리
TF-IDF (Term Frequency-Inverse Document Frequency)
정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되


리트리버(retriever)가 벡터 데이터베이스에서 기본적으로 수행하는 검색 유형은 유사도 검색입니다.

LangChain Vector Stores는 [Max Marginal Relevance](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html#langchain_core.vectorstores.VectorStore.max_marginal_relevance_search)를 통한 검색도 지원하므로, 이를 대신 사용하고 싶다면 다음과 같이 `search_type` 속성을 설정하면 됩니다.


- `retriever` 객체의 `search_type` 속성을 `SearchType.mmr`로 설정합니다.
  - 이는 검색 시 MMR(Maximal Marginal Relevance) 알고리즘을 사용하도록 지정하는 것입니다.


In [23]:
from langchain.retrievers.multi_vector import SearchType

# 검색 유형을 MMR(Maximal Marginal Relevance)로 설정
retriever.search_type = SearchType.mmr

# 관련 문서 전체를 검색
print(retriever.invoke("삼성전자가 만든 생성형 AI 의 이름은?")[0].page_content)

연관키워드: 자연어 처리, 딥러닝, 텍스트 생성
FAISS (Facebook AI Similarity Search)
정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리
로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수
있도록 설계되었습니다.
예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾
는 데 FAISS가 사용될 수 있습니다.
연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화
Open Source
정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용,
수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신
을 촉진하는 데 중요한 역할을 합니다.
예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.
연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업
Structured Data
정의: 구조화된 데이터는 정해진 형식이나 스키마에 따라 조직된
데이터입니다. 이는 데이터베이스, 스프레드시트 등에서 쉽게 검색
하고 분석할 수 있습니다.
예시: 관계형 데이터베이스에 저장된 고객 정보 테이블은 구조화된
데이터의 예입니다.
연관키워드: 데이터베이스, 데이터 분석, 데이터 모델링
Parser
정의: 파서는 주어진 데이터(문자열, 파일 등)를 분석하여 구조화
된 형태로 변환하는 도구입니다. 이는 프로그래밍 언어의 구문 분
석이나 파일 데이터 처리에 사용됩니다.
예시: HTML 문서를 구문 분석하여 웹 페이지의 DOM 구조를 생성하
는 것은 파싱의 한 예입니다.
연관키워드: 구문 분석, 컴파일러, 데이터 처리
TF-IDF (Term Frequency-Inverse Document Frequency)
정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되


In [26]:
!pip install -qU langchain_core

In [27]:
from langchain.retrievers.multi_vector import SearchType

# 검색 유형을 similarity_score_threshold로 설정
retriever.search_type = SearchType.similarity_score_threshold
retriever.search_kwargs = {"score_threshold": 0.3}

# 관련 문서 전체를 검색
retriever.invoke("삼성전자가 만든 생성형 AI 의 이름은?")
# print(retriever.invoke("삼성전자가 만든 생성형 AI 의 이름은?")[0].page_content)

[]

In [28]:
from langchain.retrievers.multi_vector import SearchType

# 검색 유형을 similarity로 설정, k값을 1로 설정
retriever.search_type = SearchType.similarity
retriever.search_kwargs = {"k": 1}

# 관련 문서 전체를 검색
print(len(retriever.invoke("삼성전자가 만든 생성형 AI 의 이름은?")))

1


## 요약본(summary)을 벡터저장소에 저장

요약은 종종 청크(chunk)의 내용을 보다 정확하게 추출할 수 있어 더 나은 검색 결과를 얻을 수 있습니다.

여기서는 요약을 생성하는 방법과 이를 임베딩하는 방법에 대해 설명합니다.


In [30]:
!pip install -qU pymupdf

In [32]:
# PDF 파일을 로드하고 텍스트를 분할하기 위한 라이브러리 임포트
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# PDF 파일 로더 초기화
loader = PyMuPDFLoader("/content/appendix-keywords.txt")

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=50)

# PDF 파일 로드 및 텍스트 분할 실행
split_docs = loader.load_and_split(text_splitter)

# 분할된 문서의 개수 출력
print(f"분할된 문서의 개수: {len(split_docs)}")

분할된 문서의 개수: 13


In [33]:
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


summary_chain = (
    {"doc": lambda x: x.page_content}
    # 문서 요약을 위한 프롬프트 템플릿 생성
    | ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert in summarizing documents in Korean."),
            (
                "user",
                "Summarize the following documents in 3 sentences in bullet points format.\n\n{doc}",
            ),
        ]
    )
    # OpenAI의 ChatGPT 모델을 사용하여 요약 생성
    | ChatOpenAI(temperature=0, model="gpt-4o-mini")
    | StrOutputParser()
)

`chain.batch` 메서드를 사용하여 `docs` 리스트의 문서들을 일괄 요약합니다.
- 여기서 `max_concurrency` 매개변수를 10 으로 설정하여 최대 10개의 문서를 동시에 처리할 수 있도록 합니다.

In [34]:
# 문서 배치 처리
summaries = summary_chain.batch(split_docs, {"max_concurrency": 10})

In [36]:
len(summaries)

13

요약된 내용을 출력하여 결과를 확인합니다.


In [40]:
# 원본 문서의 내용을 출력합니다.
print(split_docs[10].page_content, end="\n\n")
# 요약을 출력합니다.
print("[요약]")
print(summaries[10])

판다스 (Pandas)
정의: 판다스는 파이썬 프로그래밍 언어를 위한 데이터 분석 및 조
작 도구를 제공하는 라이브러리입니다. 이는 데이터 분석 작업을
효율적으로 수행할 수 있게 합니다.
예시: 판다스를 사용하여 CSV 파일을 읽고, 데이터를 정제하며,
다양한 분석을 수행할 수 있습니다.
연관키워드: 데이터 분석, 파이썬, 데이터 처리
GPT (Generative Pretrained Transformer)
정의: GPT는 대규모의 데이터셋으로 사전 훈련된 생성적 언어 모델
로, 다양한 텍스트 기반 작업에 활용됩니다. 이는 입력된 텍스트에
기반하여 자연스러운 언어를 생성할 수 있습니다.
예시: 사용자가 제공한 질문에 대해 자세한 답변을 생성하는 챗봇
은 GPT 모델을 사용할 수 있습니다.
연관키워드: 자연어 처리, 텍스트 생성, 딥러닝
InstructGPT
정의: InstructGPT는 사용자의 지시에 따라 특정한 작업을 수행
하기 위해 최적화된 GPT 모델입니다. 이 모델은 보다 정확하고 관
련성 높은 결과를 생성하도록 설계되었습니다.
예시: 사용자가 "이메일 초안 작성"과 같은 특정 지시를 제공하면,

[요약]
- 판다스는 파이썬을 위한 데이터 분석 및 조작 라이브러리로, CSV 파일 읽기, 데이터 정제 및 다양한 분석 작업을 효율적으로 수행할 수 있게 해줍니다.
- GPT는 대규모 데이터셋으로 사전 훈련된 생성적 언어 모델로, 입력된 텍스트에 기반하여 자연스러운 언어를 생성하며, 챗봇과 같은 텍스트 기반 작업에 활용됩니다.
- InstructGPT는 사용자의 지시에 따라 특정 작업을 수행하도록 최적화된 GPT 모델로, 보다 정확하고 관련성 높은 결과를 생성하는 데 중점을 두고 설계되었습니다.


`Chroma` 벡터 저장소를 초기화하여 자식 청크(child chunks)를 인덱싱합니다. 이때 `OpenAIEmbeddings`를 임베딩 함수로 사용합니다.

- 문서 ID를 나타내는 키로 `"doc_id"`를 사용합니다.


In [41]:
import uuid

# 요약 정보를 저장할 벡터 저장소를 생성합니다.
summary_vectorstore = Chroma(
    collection_name="summaries",
    embedding_function=OpenAIEmbeddings(model="text-embedding-3-small"),
)

# 부모 문서를 저장할 저장소를 생성합니다.
store = InMemoryStore()

# 문서 ID를 저장할 키 이름을 지정합니다.
id_key = "doc_id"

# 검색기를 초기화합니다. (시작 시 비어 있음)
retriever = MultiVectorRetriever(
    vectorstore=summary_vectorstore,  # 벡터 저장소
    byte_store=store,  # 바이트 저장소
    id_key=id_key,  # 문서 ID 키
)
# 문서 ID를 생성합니다.
doc_ids = [str(uuid.uuid4()) for _ in split_docs]

요약된 문서와 메타데이터(여기서는 생성한 요약본에 대한 `Document ID` 입니다)를 저장합니다.


In [42]:
summary_docs = [
    # 요약된 내용을 페이지 콘텐츠로 하고, 문서 ID를 메타데이터로 포함하는 Document 객체를 생성합니다.
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries)
]

요약본의 문서의 개수는 원본 문서의 개수와 일치합니다.


In [43]:
# 요약본의 문서의 개수
len(summary_docs)

13

- `retriever.vectorstore.add_documents(summary_docs)`를 통해 `summary_docs`를 벡터 저장소에 추가합니다.
- `retriever.docstore.mset(list(zip(doc_ids, docs)))`를 사용하여 `doc_ids`와 `docs`를 매핑하여 문서 저장소에 저장합니다.


In [44]:
retriever.vectorstore.add_documents(
    summary_docs
)  # 요약된 문서를 벡터 저장소에 추가합니다.

# 문서 ID와 문서를 매핑하여 문서 저장소에 저장합니다.
retriever.docstore.mset(list(zip(doc_ids, split_docs)))

`vectorstore` 객체의 `similarity_search` 메서드를 사용하여 유사도 검색을 수행합니다.


In [45]:
# 유사도 검색을 수행합니다.
result_docs = summary_vectorstore.similarity_search(
    "삼성전자가 만든 생성형 AI 의 이름은?"
)

In [46]:
# 1개의 결과 문서를 출력합니다.
print(result_docs[0].page_content)

- HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리로, 연구자와 개발자들이 NLP 작업을 쉽게 수행할 수 있도록 돕습니다.
- 디지털 변환은 기술을 활용하여 기업의 서비스, 문화, 운영을 혁신하고 비즈니스 모델을 개선하여 경쟁력을 높이는 과정입니다.
- 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정으로, 주로 검색 엔진 최적화나 데이터 분석에 사용됩니다.


`retriever` 객체의 `invoke()` 사용하여 질문과 관련된 문서를 검색합니다.


In [47]:
# 관련된 문서를 검색하여 가져옵니다.
retrieved_docs = retriever.invoke("삼성전자가 만든 생성형 AI 의 이름은?")
print(retrieved_docs[0].page_content)

정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모
델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이
쉽게 NLP 작업을 수행할 수 있도록 돕습니다.
예시: HuggingFace의 Transformers 라이브러리를 사용하여 감
정 분석, 텍스트 생성 등의 작업을 수행할 수 있습니다.
연관키워드: 자연어 처리, 딥러닝, 라이브러리
Digital Transformation
정의: 디지털 변환은 기술을 활용하여 기업의 서비스, 문화, 운영
을 혁신하는 과정입니다. 이는 비즈니스 모델을 개선하고 디지털
기술을 통해 경쟁력을 높이는 데 중점을 둡니다.
예시: 기업이 클라우드 컴퓨팅을 도입하여 데이터 저장과 처리를
혁신하는 것은 디지털 변환의 예입니다.
연관키워드: 혁신, 기술, 비즈니스 모델
Crawling
정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를
수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자
주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠
를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진
Word2Vec


## 가설 쿼리(Hypothetical Queries) 를 활용하여 문서 내용 탐색

LLM은 특정 문서에 대해 가정할 수 있는 질문 목록을 생성하는 데에도 사용될 수 있습니다.

이렇게 생성된 질문들은 임베딩(embedding)될 수 있으며, 이를 통해 문서의 내용을 더욱 깊이 있게 탐색하고 이해할 수 있습니다.

가정 질문 생성은 문서의 주요 주제와 개념을 파악하는 데 도움이 되며, 독자들이 문서 내용에 대해 더 많은 궁금증을 갖도록 유도할 수 있습니다.


아래는 `Function Calling` 을 활용하여 가설 질문을 생성하는 예제입니다.

In [48]:
functions = [
    {
        "name": "hypothetical_questions",  # 함수의 이름을 지정합니다.
        "description": "Generate hypothetical questions",  # 함수에 대한 설명을 작성합니다.
        "parameters": {  # 함수의 매개변수를 정의합니다.
            "type": "object",  # 매개변수의 타입을 객체로 지정합니다.
            "properties": {  # 객체의 속성을 정의합니다.
                "questions": {  # 'questions' 속성을 정의합니다.
                    "type": "array",  # 'questions'의 타입을 배열로 지정합니다.
                    "items": {
                        "type": "string"
                    },  # 배열의 요소 타입을 문자열로 지정합니다.
                },
            },
            "required": ["questions"],  # 필수 매개변수로 'questions'를 지정합니다.
        },
    }
]

`ChatPromptTemplate`을 사용하여 주어진 문서를 기반으로 3개의 가상 질문을 생성하는 프롬프트 템플릿을 정의합니다.

- `functions`와 `function_call`을 설정하여 가상 질문 생성 함수를 호출합니다.
- `JsonKeyOutputFunctionsParser`를 사용하여 생성된 가상 질문을 파싱하고, `questions` 키에 해당하는 값을 추출합니다.

In [49]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
from langchain_openai import ChatOpenAI

hypothetical_query_chain = (
    {"doc": lambda x: x.page_content}
    # 아래 문서를 사용하여 답변할 수 있는 가상의 질문을 정확히 3개 생성하도록 요청합니다. 이 숫자는 조정될 수 있습니다.
    | ChatPromptTemplate.from_template(
        "Generate a list of exactly 3 hypothetical questions that the below document could be used to answer. "
        "Potential users are those interested in the AI industry. Create questions that they would be interested in. "
        "Output should be written in Korean:\n\n{doc}"
    )
    | ChatOpenAI(max_retries=0, model="gpt-4o-mini").bind(
        functions=functions, function_call={"name": "hypothetical_questions"}
    )
    # 출력에서 "questions" 키에 해당하는 값을 추출합니다.
    | JsonKeyOutputFunctionsParser(key_name="questions")
)

문서에 대한 답변을 출력합니다.

- 출력은 생성한 3개의 가설 쿼리(Hypothetical Queries) 가 담겨 있습니다.


In [51]:
# 주어진 문서에 대해 체인을 실행합니다.
hypothetical_query_chain.invoke(split_docs[10])

['판다스를 사용하여 데이터 분석 작업을 더 효율적으로 수행할 수 있는 방법은 무엇인가요?',
 'GPT 모델이 챗봇 개발에 어떻게 활용될 수 있는지 구체적인 예를 들어 설명할 수 있나요?',
 'InstructGPT을 활용하여 특정한 작업을 빠르고 정확하게 수행하기 위한 전략은 무엇인가요?']

`chain.batch` 메서드를 사용하여 `split_docs` 데이터에 대해 동시에 여러 개의 요청을 처리합니다.

In [52]:
# 문서 목록에 대해 가설 질문을 배치 생성
hypothetical_questions = hypothetical_query_chain.batch(
    split_docs, {"max_concurrency": 10}
)

In [54]:
hypothetical_questions[10]

['판다스를 사용하여 데이터 분석을 수행할 때 주요 이점은 무엇인가요?',
 'GPT 모델이 실제 산업에서 어떻게 활용될 수 있습니까?',
 'InstructGPT의 사용이 기존 GPT 모델에 비해 가지는 장점은 무엇인가요?']

아래는 이전에 진행했던 방식과 동일하게 생성한 가설 쿼리(Hypothetical Queries) 를 벡터저장소에 저장하는 과정입니다.


In [55]:
# 자식 청크를 인덱싱하는 데 사용할 벡터 저장소
hypothetical_vectorstore = Chroma(
    collection_name="hypo-questions", embedding_function=OpenAIEmbeddings()
)
# 부모 문서의 저장소 계층
store = InMemoryStore()

id_key = "doc_id"
# 검색기 (시작 시 비어 있음)
retriever = MultiVectorRetriever(
    vectorstore=hypothetical_vectorstore,
    byte_store=store,
    id_key=id_key,
)
doc_ids = [str(uuid.uuid4()) for _ in split_docs]  # 문서 ID 생성

`question_docs` 리스트에 메타데이터(문서 ID) 를 추가합니다.


In [56]:
question_docs = []
# hypothetical_questions 저장
for i, question_list in enumerate(hypothetical_questions):
    question_docs.extend(
        # 질문 리스트의 각 질문에 대해 Document 객체를 생성하고, 메타데이터에 해당 질문의 문서 ID를 포함시킵니다.
        [Document(page_content=s, metadata={id_key: doc_ids[i]}) for s in question_list]
    )

가설 쿼리를 문서에 추가하고, 원본 문서를 `docstore` 에 추가합니다.


In [57]:
# hypothetical_questions 문서를 벡터 저장소에 추가합니다.
retriever.vectorstore.add_documents(question_docs)

# 문서 ID와 문서를 매핑하여 문서 저장소에 저장합니다.
retriever.docstore.mset(list(zip(doc_ids, split_docs)))

`vectorstore` 객체의 `similarity_search` 메서드를 사용하여 유사도 검색을 수행합니다.


In [58]:
# 유사한 문서를 벡터 저장소에서 검색합니다.
result_docs = hypothetical_vectorstore.similarity_search(
    "삼성전자가 만든 생성형 AI 의 이름은?"
)

아래는 유사도 검색 결과입니다.

여기서는 생성한 가설 쿼리만 추가해 놓은 상태이기 때문에, 생성한 가설 쿼리 중 유사도가 가장 높은 문서를 반환합니다.


In [59]:
# 유사도 검색 결과를 출력합니다.
for doc in result_docs:
    print(doc.page_content)
    print(doc.metadata)

벡터스토어를 활용하여 AI 모델의 검색 효율성을 높이는 방법은 무엇인가요?
{'doc_id': '2070586e-482d-44a1-82e7-6730f1620df2'}
사용자가 입력한 키워드에 따라 인공지능이 정보를 더 잘 식별하고 제공하기 위해 어떤 방법들이 사용될 수 있을까요?
{'doc_id': '6a68eb12-b8b8-4480-801e-3bcdadb196e6'}
자연어 처리와 벡터스토어의 결합이 AI 산업에 미치는 영향은 어떤 것들이 있을까요?
{'doc_id': '2070586e-482d-44a1-82e7-6730f1620df2'}
인공지능을 활용한 자연어 처리 기술의 발전이 검색 엔진의 결과 순위에 미치는 영향은 무엇인가요?
{'doc_id': '6a68eb12-b8b8-4480-801e-3bcdadb196e6'}


`retriever` 객체의 `invoke` 메서드를 사용하여 쿼리와 관련된 문서를 검색합니다.


In [60]:
# 관련된 문서를 검색하여 가져옵니다.
retrieved_docs = retriever.invoke(result_docs[1].page_content)

# 검색된 문서를 출력합니다.
for doc in retrieved_docs:
    print(doc.page_content)

예시: 사용자가 "이메일 초안 작성"과 같은 특정 지시를 제공하면,
InstructGPT는 관련 내용을 기반으로 이메일을 작성합니다.
연관키워드: 인공지능, 자연어 이해, 명령 기반 처리
Keyword Search
정의: 키워드 검색은 사용자가 입력한 키워드를 기반으로 정보를
찾는 과정입니다. 이는 대부분의 검색 엔진과 데이터베이스 시스템
에서 기본적인 검색 방식으로 사용됩니다.
예시: 사용자가 "커피숍 서울"이라고 검색하면, 관련된 커피숍 목
록을 반환합니다.
연관키워드: 검색 엔진, 데이터 검색, 정보 검색
Page Rank
정의: 페이지 랭크는 웹 페이지의 중요도를 평가하는 알고리즘으
로, 주로 검색 엔진 결과의 순위를 결정하는 데 사용됩니다. 이는
데이터의 예입니다.
연관키워드: 데이터베이스, 데이터 분석, 데이터 모델링
Parser
정의: 파서는 주어진 데이터(문자열, 파일 등)를 분석하여 구조화
된 형태로 변환하는 도구입니다. 이는 프로그래밍 언어의 구문 분
석이나 파일 데이터 처리에 사용됩니다.
예시: HTML 문서를 구문 분석하여 웹 페이지의 DOM 구조를 생성하
는 것은 파싱의 한 예입니다.
연관키워드: 구문 분석, 컴파일러, 데이터 처리
TF-IDF (Term Frequency-Inverse Document Frequency)
정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되
다. 이는 자연어 처리에서 데이터를 전처리하는 데 사용됩니다.
예시: "I love programming."이라는 문장을 ["I", "love",
"programming", "."]으로 분할합니다.
연관키워드: 토큰화, 자연어 처리, 구문 분석
VectorStore
정의: 벡터스토어는 벡터 형식으로 변환된 데이터를 저장하는 시스
템입니다. 이는 검색, 분류 및 기타 데이터 분석 작업에 사용됩니
다.
예시: 단어 임베딩 벡터들을 데이터베이스에 저장하여 빠르게 접근
